# 🟦 アソシエーション分析

「AならばBである」という関係を抽出する

In [4]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# ファイルを1行ずつ読み込む（各行が1トランザクション）
with open("groceries.csv", "r", encoding="utf-8") as f:
    transactions = [line.strip().split(',') for line in f.readlines()]

# One-Hotエンコーディング
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# 頻出アイテムセット抽出
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)
print("頻出アイテムセット:")
print(frequent_itemsets.head())

# アソシエーションルール生成
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
print("\nアソシエーションルール（上位5件）:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())


頻出アイテムセット:
    support        itemsets
0  0.033452      (UHT-milk)
1  0.052466          (beef)
2  0.033249       (berries)
3  0.026029     (beverages)
4  0.080529  (bottled beer)

アソシエーションルール（上位5件）:
          antecedents         consequents   support  confidence      lift
0        (whole milk)              (beef)  0.021251    0.083168  1.585180
1              (beef)        (whole milk)  0.021251    0.405039  1.585180
2     (bottled water)  (other vegetables)  0.024809    0.224471  1.160101
3  (other vegetables)     (bottled water)  0.024809    0.128219  1.160101
4     (bottled water)        (rolls/buns)  0.024199    0.218951  1.190373


### 🟠 apriori関数

トランザクションデータ（買い物かごデータなど）から、一定以上の頻度で一緒に出現するアイテムの組み合わせ（頻出アイテムセット）を抽出する関数。  

```python
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)
```

* df
  * One-Hotエンコーディングされたデータフレーム。
  * 各列が「商品名」、各行が「トランザクション（取引）」を表す。
  * 値は True / False（購入したかどうか）。
* min_support=0.02
  * 最小サポート値。
  * 「全トランザクションのうち2%以上で出現するアイテムセット」を抽出。
  * 値を小さくするとレアな組み合わせも出るが、計算量が増える。
* use_colnames=True
  * True にすると、結果のアイテムセットに列名（商品名）を表示。
  * False の場合は列インデックス番号になる。


### 🟠 association_rules関数

aprioriで得られた頻出アイテムセットから、「もしAを買ったらBも買う可能性が高い」というアソシエーションルールを導出する関数。

```python
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
```

* frequent_itemsets
  * `apriori` で求めた頻出アイテムセット。
* metric="lift"
  * ルールを評価する指標の種類。
  * `"support"`, `"confidence"`, `"lift"`, `"leverage"`, `"conviction"` などから選べる。
  * ここでは「リフト値」でフィルタする。
* min_threshold=1.0
  * metric の最小値。
  * lift が 1.0 以上（＝正の相関がある）ルールのみを抽出。


### 🟠 出力の見方

#### 頻出アイテムセット（例）

| support | itemsets        |
| ------- | --------------- |
| 0.25    | {milk}          |
| 0.15    | {bread, butter} |

* **support（サポート）**
  全取引の中でそのアイテムセットが出現する割合。
  例: 0.25 → 全体の25%の取引に含まれる。

---

#### アソシエーションルール（例）

| antecedents | consequents | support | confidence | lift |
| ----------- | ----------- | ------- | ---------- | ---- |
| {bread}     | {butter}    | 0.15    | 0.6        | 1.5  |

* **antecedents**：前件（もし〜なら）
* **consequents**：後件（〜が起きる）
* **support**：両方が同時に出現する割合
* **confidence**：前件が出たときに後件も出る確率（条件付き確率）
* **lift**：独立の場合と比べた相関の強さ

  * lift > 1：正の相関（同時に出やすい）
  * lift = 1：独立
  * lift < 1：負の相関（同時に出にくい）


